In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
import seaborn as sns
from tensorflow.keras import backend as K
from tensorflow.python.framework.ops import disable_eager_execution
import drive.MyDrive.Petrocchi.util.preprocessing as preprocessing
from tensorflow.keras import models
from tensorflow.python.util import deprecation

deprecation._PRINT_DEPRECATION_WARNINGS = False
disable_eager_execution() #To execute the gradient

In [ ]:
def plot_history(history):
  '''
  Plots Training and validation loss and accuracy graphs
  '''
  acc = history['accuracy']
  val_acc = history['val_accuracy']
  loss = history['loss']
  val_loss = history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, '--b.', label='Training accuracy')
  plt.plot(epochs, val_acc, '--r.', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.grid(color='grey', which='major', axis='y', linestyle='--')

  plt.figure()

  plt.plot(epochs, loss, '--b.', label='Training loss')
  plt.plot(epochs, val_loss, '--r.', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.grid(color='grey', which='major', axis='y', linestyle='--')

  plt.show()

print('"plot_history" function loaded' )

"plot_history" function loaded


In [ ]:
def plot_roc_curve(test_labels, models_test_predictions, model_names):
  '''
  Plot a roc curve for every model predictions 
  '''
  color_idxs = np.linspace(0, 1, len(model_names))
  for test_predictions, name, color_idx in zip(models_test_predictions, model_names, color_idxs):
    fpr, tpr, _ = roc_curve(test_labels, test_predictions)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=plt.cm.winter(color_idx),
            lw=2, label= name + ' (area = %0.3f)' % roc_auc)
    
  plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver operating characteristic')
  plt.legend(loc="lower right")
  plt.show()

print('"plot_roc_curve" function loaded' )

"plot_roc_curve" function loaded


In [ ]:
def plot_confusion_matrix(test_labels, test_predictions, labels_names):
  '''
  Plot a beautiful confusion matrix
  '''
  c_matrix = confusion_matrix(test_labels, test_predictions)
  c_matrix = c_matrix.transpose()
  group_names = ['T. ' + labels_names[0],'F. ' + labels_names[0],'F. ' + labels_names[1],'T. ' + labels_names[1]]
  group_counts = ['{0:0.0f}'.format(value) for value in
                  c_matrix.flatten()]
  precision_percentages = ['Prec: ' + '{0:.2%}'.format(c_matrix[0,0]/np.sum(c_matrix[0])), 
                       'Prec: ' + '{0:.2%}'.format(c_matrix[0,1]/np.sum(c_matrix[0])),
                       'Prec: ' + '{0:.2%}'.format(c_matrix[1,0]/np.sum(c_matrix[1])),
                       'Prec: ' + '{0:.2%}'.format(c_matrix[1,1]/np.sum(c_matrix[1]))]
  recall_percentages = ['Rec: ' + '{0:.2%}'.format(c_matrix[0,0]/np.sum(c_matrix[:,0])), 
                       'Rec: ' + '{0:.2%}'.format(c_matrix[0,1]/np.sum(c_matrix[:,1])),
                       'Rec: ' + '{0:.2%}'.format(c_matrix[1,0]/np.sum(c_matrix[:,0])),
                       'Rec: ' + '{0:.2%}'.format(c_matrix[1,1]/np.sum(c_matrix[:,1]))]
  labels = [f'{v1}\n{v2}\n{v3}\n{v4}' for v1, v2, v3, v4 in
            zip(group_names,group_counts,precision_percentages,recall_percentages)]
  labels = np.asarray(labels).reshape(2,2)
  yticks = ["PREDICTED\n"+labels_names[0],"PREDICTED\n"+labels_names[1]]
  sns.heatmap(c_matrix, annot=labels, fmt='', cmap='Blues' ,  xticklabels=["ACTUAL " + labels_names[0], "ACTUAL " + labels_names[1]], yticklabels=yticks)
  plt.yticks(rotation=0, ) 
  plt.show()

print('"plot_confusion_matrix" function loaded' )

"plot_confusion_matrix" function loaded


In [ ]:
def generate_pattern(model, layer, filter_index, size, grad_steps):
    ''' 
    Build a loss function that maximizes the activation
    of the layer's filter considered
    '''
    layer_output = model.layers[layer].output
    loss = K.mean(layer_output[:, :, :, filter_index])

    # Compute the gradient of the loss wrt the input picture
    grads = K.gradients(loss, model.input)[0]

    # Normalization trick: we normalize the gradient
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

    # This function returns the loss and grads given the input picture
    iterate = K.function([model.input], [loss, grads])
    
    # We start from a gray image with some noise
    input_img_data = np.random.random((1, size, size, 1)) * 20.

    # Run gradient ascent for grad_steps steps
    for i in range(grad_steps):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value
        
    return preprocessing.deprocess_image(input_img_data[0,:,:,:])[:,:,0]

print('"generate_pattern" function loaded' )

"generate_pattern" function loaded


In [ ]:
def display_filters_patterns(top_k, model, size, images_per_row, scale, grad_steps):
  '''
  Show max activation images for all filters in top_k layers
  '''
  for layer in range(top_k):
      # This is the number of filters in the layer
      n_filters = model.layers[layer].output.shape[3]

      # Patterns are tiled in this matrix
      n_cols = n_filters // images_per_row
      display_grid = np.zeros((size * n_cols, images_per_row * size))

      # Patterns are tiled into this big horizontal grid
      for col in range(n_cols):
          for row in range(images_per_row):
              display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = generate_pattern(model,layer,col * images_per_row + row,size, grad_steps)

      # Display the grid
      final_scale = (1./size)*scale # to fit in the screen
      plt.figure(figsize=(final_scale * display_grid.shape[1],
                          final_scale * display_grid.shape[0]))
      plt.title(model.layers[layer].name)
      plt.grid(False)
      plt.imshow(display_grid, aspect='auto', cmap='gray')
      plt.show()

print('"display_filters_patterns" function loaded' )

"display_filters_patterns" function loaded


In [ ]:
def display_features_maps(bottom_layer, top_k, model, example_image, images_per_row, scale):
  '''
  Show all features maps from the elaboration of "example_image" image in top_k layers
  '''
  # Creates a model that will return these outputs, given the model input:
  layer_outputs = [layer.output for layer in model.layers[bottom_layer:top_k + 1]]
  activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
  activations = activation_model.predict(example_image)

  # Display the feature maps
  for layer in range(bottom_layer, top_k):
      # This is the number of features in the feature map
      n_features = activations[layer].shape[3]

      # The feature map has shape (1, size, size, n_features)
      size = activations[layer].shape[1]

      # Tiles the activation channels in this matrix
      n_cols = n_features // images_per_row
      display_grid = np.zeros((size * n_cols, images_per_row * size))

      # Tiles each filter into this big horizontal grid
      for col in range(n_cols):
          for row in range(images_per_row):
              channel_image = activations[layer][0,:,:,col * images_per_row + row]
              # Post-process the feature to make it visually palatable
              channel_image = preprocessing.deprocess_image(channel_image)
              display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image

      # Display the grid
      final_scale = (1./size)*scale # to fit the screen
      plt.figure(figsize=(final_scale * display_grid.shape[1],
                          final_scale * display_grid.shape[0]))
      plt.title(model.layers[layer].name)
      plt.grid(False)
      plt.imshow(display_grid, aspect='auto', cmap='gray')
      plt.show()

print('"display_features_maps" function loaded' )

"display_features_maps" function loaded


In [ ]:
def generate_pattern_RGB(model, layer, filter_index, size, grad_steps):
    ''' 
    Build a loss function that maximizes the activation
    of the layer's filter considered
    '''
    layer_output = model.layers[layer].output
    loss = K.mean(layer_output[:, :, :, filter_index])

    # Compute the gradient of the loss wrt the input picture
    grads = K.gradients(loss, model.input)[0]

    # Normalization trick: we normalize the gradient
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

    # This function returns the loss and grads given the input picture
    iterate = K.function([model.input], [loss, grads])
    
    # We start from a gray image with some noise
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.

    # Run gradient ascent for grad_steps steps
    for i in range(grad_steps):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value
        
    return preprocessing.deprocess_image_to_RGB(input_img_data[0,:,:,:])

print('"generate_pattern_RGB" function loaded' )

In [ ]:
def display_filters_patterns_RGB(top_k, model, size, images_per_row, scale, grad_steps):
  '''
  Show max activation images for all filters in top_k layers
  '''
  for layer in range(1,top_k):
      # This is the number of filters in the layer
      n_filters = model.layers[layer].output.shape[3]

      # Patterns are tiled in this matrix
      n_cols = n_filters // images_per_row
      display_grid = np.zeros((size * n_cols, images_per_row * size, 3))

      # Patterns are tiled into this big horizontal grid
      for col in range(n_cols):
          for row in range(images_per_row):
              display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = generate_pattern_RGB(model,layer,col * images_per_row + row,size, grad_steps)

      # Display the grid
      final_scale = (1./size)*scale # to fit in the screen
      plt.figure(figsize=(final_scale * display_grid.shape[1],
                          final_scale * display_grid.shape[0]))
      plt.title(model.layers[layer].name)
      plt.grid(False)
      plt.imshow(preprocessing.deprocess_image_to_RGB(display_grid), aspect='auto')
      plt.show()

print('"display_filters_patterns" function loaded' )